# Install dependecies

In [1]:
!pip install tensorflow==2.4.1 opencv-python matplotlib

  Using cached tensorflow-2.4.1-cp38-cp38-macosx_10_11_x86_64.whl (173.9 MB)
  Using cached opencv_python-4.5.5.64-cp36-abi3-macosx_10_15_x86_64.whl (46.3 MB)
  Using cached matplotlib-3.5.2-cp38-cp38-macosx_10_9_x86_64.whl (7.3 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     |████████████████████████████████| 15.6 MB 3.9 MB/s eta 0:00:01     |██████████████████████████      | 12.7 MB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 25.2 MB/s eta 0:00:01     |█████████████████████████████   | 5.2 MB 25.2 MB/s eta 0:00:01
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached wheel-0.37.1-py2.py3-none-any.whl (35 kB)
     |████████████████████████████████| 132 kB 32.1 MB/s eta 0:00:01
  Using cached protobuf-3.20.1-cp38-cp38-macosx_10_9_x86_64.whl (962 kB)
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
     |████████████████████████████████| 462 kB 14.8 MB/s eta 

# Import dependencies

In [2]:
#standard dependencies
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [3]:
#tensorflow dependencies- Functional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

In [4]:
#Setup paths
POS_PATH = os.path.join('data','positive')
NEG_PATH = os.path.join('data','negative')
ANC_PATH = os.path.join('data','anchor')

In [6]:
#making directories
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)